In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [31]:
stock_prices = pd.read_csv("../stock_prices.csv")
len(stock_prices)

2332531

In [32]:
secondary_stock_prices = pd.read_csv("../secondary_stock_prices.csv")
supplemental_prices = pd.read_csv("../supplimental_stock_prices.csv")
supplemental_secondary_stock_prices = pd.read_csv("../supplimental_secondary_stock_prices.csv")
stock_prices = stock_prices.append(secondary_stock_prices)
stock_prices = stock_prices.append(supplemental_prices)
stock_prices = stock_prices.append(supplemental_secondary_stock_prices)


/var/folders/df/hv6t4fkd2tq0qjflfg5wj3cw0000gr/T/ipykernel_2874/2119386523.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stock_prices = stock_prices.append(secondary_stock_prices)
/var/folders/df/hv6t4fkd2tq0qjflfg5wj3cw0000gr/T/ipykernel_2874/2119386523.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stock_prices = stock_prices.append(supplemental_prices)
/var/folders/df/hv6t4fkd2tq0qjflfg5wj3cw0000gr/T/ipykernel_2874/2119386523.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stock_prices = stock_prices.append(supplemental_secondary_stock_prices)


In [33]:
stock_prices.head()

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730
1,20170104_1332,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,1.0,NaN,False,0.012324
2,20170104_1333,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800,1.0,NaN,False,0.006154
3,20170104_1376,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300,1.0,NaN,False,0.011053
4,20170104_1377,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800,1.0,NaN,False,0.003026


In [34]:
stock_prices.isna().sum()

RowId                     0
Date                      0
SecuritiesCode            0
Open                 110826
High                 110826
Low                  110826
Close                110826
Volume                    0
AdjustmentFactor          0
ExpectedDividend    5230197
SupervisionFlag           0
Target                 1046
dtype: int64

In [35]:
from scipy import stats
def pre_process(data):
    data['ExpectedDividend'] = data['ExpectedDividend'].fillna(0)
    data["SupervisionFlag"] = data["SupervisionFlag"].astype(int)
    data['Target'] = data['Target'].fillna(0)
    
    cols = ['Open', 'High', 'Low', 'Close']
    data.loc[:,cols] = data.loc[:,cols].ffill()
    data.loc[:,cols] = data.loc[:,cols].bfill()

    data['Daily_Range'] = data['Close'] - data['Open']
    
    data['Open'] = stats.zscore(data['Open'])
    data['High'] = stats.zscore(data['High'])
    data['Low'] = stats.zscore(data['Low'])
    data['Close'] = stats.zscore(data['Close'])
    data['Volume'] = stats.zscore(data['Volume'])
    data['Daily_Range'] = stats.zscore(data['Daily_Range'])
    
    data = data.drop(['RowId', 'Date'], axis=1)
    
    return data

In [36]:
data = pre_process(stock_prices)

In [37]:
X_data = data.drop(['Target'], axis=1)
y_data = data['Target']

In [38]:
print(X_data.head())
print(y_data.head())

   SecuritiesCode      Open      High       Low     Close    Volume  \
0            1301 -0.089035 -0.089252 -0.088276 -0.088774 -0.129272   
1            1332 -0.150313 -0.150384 -0.150109 -0.150198  0.709902   
2            1333 -0.077267 -0.076487 -0.076577 -0.075533 -0.056670   
3            1376 -0.123663 -0.123059 -0.123087 -0.122499 -0.135368   
4            1377 -0.073872 -0.072559 -0.072867 -0.072138 -0.093062   

   AdjustmentFactor  ExpectedDividend  SupervisionFlag  Daily_Range  
0               1.0               0.0                0     0.021115  
1               1.0               0.0                0     0.009975  
2               1.0               0.0                0     0.136975  
3               1.0               0.0                0     0.092413  
4               1.0               0.0                0     0.136975  
0    0.000730
1    0.012324
2    0.006154
3    0.011053
4    0.003026
Name: Target, dtype: float64


In [39]:
from sklearn.linear_model import LinearRegression

In [40]:
model = LinearRegression()

In [41]:
from sklearn.model_selection import train_test_split
x_train, x_test,y_train,y_test = train_test_split(X_data,y_data,test_size =0.2)

In [42]:
model.fit(x_train, y_train)

LinearRegression()

In [43]:
model.score(x_test, y_test)

0.000172221056336741